# Currency exchange rates

For spend-based emission factors, we define the currencies supported by the climatiq API in the `pint` unit registry.

For this, current exchange rates are extracted.

Note: OCEAn currently does not actively convert any currencies. However, they do need to be defined somehow for the user to assign a currency to an OCEL attribute. When accessing a climatiq emission factor, the original unit is passed, with any conversions then handled by the climatiq API.

In [ ]:
import os
from pathlib import Path
os.chdir("../../src/backend")
assert Path(os.getcwd()).name == "backend"

import re
import pandas as pd
import pint

from api.config import config
from units.climatiq import ClimatiqUnitType

## How to update exchange rates?

- Below, set `CURRENCY_EXCHANGE_DATE` to today and execute the cell.

In [ ]:
# CURRENCY_EXCHANGE_DATE = "20240314"
CURRENCY_EXCHANGE_DATE = "20241005"

ref = "eur"
"""Reference currency"""

PINT_CURRENCY_CONTEXT = f"FX_{CURRENCY_EXCHANGE_DATE}"
CSV_FILE_NAME = f"{CURRENCY_EXCHANGE_DATE}_currency_exchange_rates.csv"
UNIT_DEFINITION_FILE_PATH = config.DATA_DIR /"units" / f"{CURRENCY_EXCHANGE_DATE}_climatiq_pint_currencies.txt"

print(f"CURRENCY_EXCHANGE_DATE: {CURRENCY_EXCHANGE_DATE}")
print(f"PINT_CURRENCY_CONTEXT: {PINT_CURRENCY_CONTEXT}")
print(f"CSV_FILE_NAME: {CSV_FILE_NAME}")

- Copy a spreadsheet named like `YYYYMMDD_currency_exchange_rates.xlsx` and rename with date equal to `CURRENCY_EXCHANGE_DATE`.
- Open the file with Excel. The exchange rates should update automatically. To check this, compare with old `.csv` exports.
- Save as `.csv` with name like `CSV_FILE_NAME`.
- Run the following sanity checks.

In [ ]:
cdf = pd.read_csv(config.DATA_DIR /"units" / CSV_FILE_NAME, delimiter=";", decimal=",")
cdf["iso_currency_code"] = cdf["iso_currency_code"].str.lower()
cdf["rel"] = cdf["rel"].str.lower()
cdf.loc[cdf["rel"] == f"{ref}/{ref}", "rate"] = 1

# Completeness checks for conversion table
climatiq_currencies = ClimatiqUnitType._UNIT_TYPES["Money"].units["money_unit"]
missing_currencies = [c for c in climatiq_currencies if c.lower() not in cdf["iso_currency_code"].values]
assert ref in cdf["iso_currency_code"].values, f"Reference currency ({ref}) not found."
assert not missing_currencies, f"Conversion table misses {len(missing_currencies)} currencies ({', '.join(missing_currencies)})"
assert cdf.apply(lambda row: row["rel"] == f"{row['iso_currency_code']}/{ref}", axis=1).all(), f"Inconsistent currency rates (must refer to {ref})"
assert cdf.drop(columns=["currency", "country", "symbol"]).notna().all().all(), "Conversion table has empty cells"

cdf

## Export and use

- Run the following cells to create a new pint currency context file.
- In `.env`, update `CURRENCY_EXCHANGE_DATE`.

In [ ]:
def make_unit_definition_file(cdf: pd.DataFrame):

    def get_alias_string(row):
        # symbol = row["symbol"] if not pd.isna(row["symbol"]) else "_"
        # Use iso code as symbol too
        iso = row["iso_currency_code"]
        symbol = iso
        name_alias = row["currency"].replace(" ", "")
        aliases = [symbol, name_alias]
        return "".join([f" = {a}" for a in aliases if a])
    
    lines = []
    ref_row = cdf[cdf["iso_currency_code"] == ref].to_dict("records")[0]
    lines.append(f"{ref} = [currency]" + get_alias_string(ref_row))
    lines.append("")
    
    # https://pint.readthedocs.io/en/stable/advanced/currencies.html
    # https://pint.readthedocs.io/en/stable/user/contexts.html
    
    # Define currency names, without conversion rates
    for i, row in cdf.iterrows():
        iso = row["iso_currency_code"]
        if iso == ref:
            continue
        aliases = [row["currency"], row["symbol"]]
        lines.append(f"{iso} = nan {ref}" + get_alias_string(row))
    
    # Set conversion rates inside a context
    lines.append("")
    lines.append(f"@context {PINT_CURRENCY_CONTEXT}")
    for i, row in cdf.iterrows():
        iso = row["iso_currency_code"]
        if iso == ref:
            continue
        rate = row["rate"]
        lines.append(f"    {iso} = {rate} {ref}")
    lines.append("@end")
    return lines

lines = make_unit_definition_file(cdf)
for s in lines:
    print(s)

In [ ]:
with open(UNIT_DEFINITION_FILE_PATH, "w", encoding="utf8") as f:
    f.write("\n".join(make_unit_definition_file(cdf)))
print(f"Custom climatiq units saved to {UNIT_DEFINITION_FILE_PATH}.")

In [ ]:
import pint

ureg = pint.UnitRegistry()
ureg.load_definitions(UNIT_DEFINITION_FILE_PATH)
ureg.enable_contexts(PINT_CURRENCY_CONTEXT)

ureg("2.5 eur").to("usd").to_base_units()